In [1]:
import time
import math
import sympy as sp
import numpy as np
from numpy.polynomial import polynomial as p
import matplotlib.pyplot as plt
from sage.all import *
from sage.stats.distributions.discrete_gaussian_integer import DiscreteGaussianDistributionIntegerSampler
from sage.stats.distributions.discrete_gaussian_polynomial import DiscreteGaussianDistributionPolynomialSampler 

In [2]:
def balance(x, q):
    return np.array([n if n <= (q-1)/2 else n-q for n in x.flatten()]).reshape(np.shape(x))

In [3]:
class commitmentScheme(object):
    lamb = None
    p = None
    n = None
    k = None 
    m = None
    l = None
    v = None
    N = None
    B = None
    
    def __init__(self, lamb, p, n, k, m):
        self.lamb = lamb
        self.p = p
        self.n = n 
        self.k = k 
        self.m = m
        
    def gen(self):
        #Set parameters as in BBC+18
        P = self.n*self.k**2*self.m**2*self.p**2
        B = P*self.k*self.m*self.n #B = O(PN)
        Pp = B*P
        r = int(np.ceil(np.log(self.n*self.k*self.m))) #r = O(log n) or O(log N)? 
        q = sp.nextprime(Pp*np.sqrt(r)) #y u so big?
        print("P = " + str(P))
        print("B = " + str(B))
        print("Pp = " + str(Pp))
        print("r = " + str(r))
        print("log2 q = " + str(math.log2(q)))
        
        #n = int(np.ceil(np.sqrt(self.N*r*math.log(q, self.p))))
        #k = int(np.ceil(self.lamb/np.log2(self.p)))
        #m = int(np.ceil(self.N/(n*k)))
        
        A1 = np.array([randint((-q+1)//2, (q-1)//2) for _ in range(r*int(2*r*math.log(q, self.p)))]).reshape(r, int(2*r*math.log(q, self.p))) 
        A2 = np.array([randint((-q+1)//2, (q-1)//2) for _ in range(r*self.n)]).reshape(r, self.n)
        print("Shapes of matrices A1, A2: " + str(A1.shape) + " " + str(A2.shape))
        return (self.p, q, r, A1, A2)
    
    def commit(self, msg, r, ck): #r = randomness
        q = ck[1]
        A1 = ck[3]
        A2 = ck[4]
        C = np.matmul(A1, r[0])%q + np.matmul(A2, msg[0])%q
        for i in range(self.k-1):
            C = np.vstack((C, np.matmul(A1, r[i+1])%q + np.matmul(A2, msg[i+1])%q))
        return balance(C, q)
        #return C

In [8]:
def commit_test():
    
    #Commitment for k messages msg in (Z_p)^n
    #Can be seen as a commitment for a (nxk)-matrix
    #In the arithmetic circuit protocol we will commit to m (nxk)-matrices 
    
    lamb = 128
    p = 4099
    #N = 10
    n = 100
    k = int(np.ceil(lamb/np.log2(p)))
    m = 10 #int(N/(k*n))
    
    sigma1 = 48*np.sqrt(k*n)*k*m*p**2
    
    D = DiscreteGaussianDistributionIntegerSampler(sigma1)
    
    msg = np.random.randint((-p+1)/2, (p-1)/2+1, size = [k, n])
    #msg = np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    
    print("Messages: " + str(msg) + " (modulo " + str(p) + ")")
    
    start_time = time.time()

    cs = commitmentScheme(lamb, p, n, k, m)

    ck = cs.gen()

    r = np.array([D() for _ in range(k*int(2*ck[2]*math.log(ck[1], ck[0])))]).reshape(k, int(2*ck[2]*math.log(ck[1], ck[0])))

    com_msg = cs.commit(msg, r, ck)
    
    end_time = time.time()

    print("sigma1 = " + str(sigma1))

    print("Committed messages: " + str(com_msg) + " (modulo " + str(ck[1]) + ")")
    
    print("Total execution time: " + str(end_time - start_time) + " seconds")

commit_test()

Messages: [[-1954 -1774  -133 ... -1773  -439  -312]
 [-1763   405   -16 ...   458  -107  -648]
 [ 1209  -266  1132 ... -1814  2048   824]
 ...
 [  -14   769 -1717 ...   646  -424 -1503]
 [ 1868 -1610 -1347 ...  -740 -1583 -1521]
 [ -216  -362  1991 ... -1101 -1373   538]] (modulo 4099)
P = 20330179210000
B = 223631971310000000
Pp = 4546478053817878465100000000000
r = 10
log2 q = 103.50355628293026
Shapes of matrices A1, A2: (10, 172) (10, 100)
sigma1 = 2942294241174.7183
Committed messages: [[961684211913977564746218767026 4942420501295613134534719992859
  -509685884843494745703784985725 -1688507878255019209658048145357
  8093906687312942535301565071118 12900838687246413703060286356633
  7131995770238240679708694217132 -3785535511833306123161663359353
  -3231379625499958407688256131064 11662399251700519793686632952990]
 [-6213412886944846294719103739037 -77843595812534433680068372096
  11063058376564732336684166390661 2633172502767275338537811240336
  9931598498540868890663082862802 -